# Summary

ConversationManager refactor attempt. Trying to change its interface so it can more effectively:
1. Support longer conversations via prompting with a subset of past responseses,
2. Support longer conversations via summarizing past conv, and
3. Still work with my GUI.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from itertools import zip_longest
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize
import numpy as np
import os
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    query_gpt3, print_response
from jabberwocky.utils import bold, colored
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [482]:
class ProtoConversationManager:
    """Similar to PromptManager but designed for ongoing conversations. This
    currently references just a single prompt: conversation.
    """

    img_exts = {'.jpg', '.jpeg', '.png'}

    def __init__(self, *names, data_dir='./data',
                 backup_image='data/misc/unknown_person.png', 
                 turn_window=4):
        assert 1 <= turn_window <= 20, 'turn_window should be in [1, 20].'
        
        # We'll be adding in the user's newest turn separately from accessing
        # their historical turns so we need to subtract 1 from both of these.
        self.user_turn_window = int(np.ceil(turn_window / 2)) - 1
        self.gpt3_turn_window = turn_window - self.user_turn_window - 1
        
        # Set directories for data storage, logging, etc.
        self.backup_image = Path(backup_image)
        self.data_dir = Path(data_dir)
        self.persona_dir = self.data_dir/'conversation_personas'
        self.conversation_dir = self.data_dir/'conversations'
        self.log_dir = self.data_dir/'logs'
        self.log_path = Path(self.log_dir)/'conversation_query_kwargs.json'
        for dir_ in (self.persona_dir, self.conversation_dir, self.log_dir):
            os.makedirs(dir_, exist_ok=True)

        # These attributes will be updated when we load a persona and cleared
        # when we end a conversation. current_persona is the processed name
        # (i.e. lowercase w/ underscores).
        self.current_persona = ''
        self.current_summary = ''
        self.current_img_path = ''
        self.current_gender = ''
        self.cached_query = ''
        self.user_turns = []
        self.gpt3_turns = []

        # Load prompt, default query kwargs, and existing personas.
        self._kwargs = load_prompt('conversation')
        self._base_prompt = self._kwargs.pop('prompt')

        # Populated by _load_personas().
        self.name2img_path = {}
        self.name2base = {}
        self.name2gender = {}
        self._load_personas(names)

    def _load_personas(self, names):
        names = names or [path.stem for path in self.persona_dir.iterdir()]
        for name in names:
            try:
                self.update_persona_dicts(self.process_name(name))
            except:
                warnings.warn(f'Could not load files for {name}.')

    def start_conversation(self, name, download_if_necessary=False):
        if name not in self:
            if not download_if_necessary:
                raise KeyError(f'{name} persona not available. You can set '
                               'download_if_necessary=True if you wish to '
                               'construct a new persona.')
            _ = self.add_persona(name, return_data=True)
        self.end_conversation()

        processed_name = self.process_name(name)
        self.current_persona = processed_name
        self.current_img_path = self.name2img_path[processed_name]
        self.current_gender = self.name2gender[processed_name]
        # This one is not returned. Info would be a bit repetitive.
        self.current_summary = self._name2summary(processed_name)
        return (self.current_persona,
                self.current_img_path,
                self.current_gender)

    def _name2summary(self, name):
        if '_' not in name: name = self.process_name(name)
        base = self.name2base[name]
        intro = sent_tokenize(base)[0]
        return base.replace(intro, '').strip()

    def end_conversation(self, fname=None):
        if fname: self.save_conversation(fname)
        self.current_summary = ''
        self.current_persona = ''
        self.current_img_path = ''
        self.current_gender = ''
        self.cached_query = ''
        self.user_turns.clear()
        self.gpt3_turns.clear()

    def save_conversation(self, fname):
        if not self.user_turns:
            raise RuntimeError('No conversation to save.')
        save(self.full_conversation, self.conversation_dir/fname)

    def add_persona(self, name, return_data=False):
        processed_name = self.process_name(name)
        dir_ = self.persona_dir/processed_name
        if dir_.exists():
            summary, img_path, gender = self.update_persona_dicts(
                processed_name, return_values=True
            )
        else:
            summary, _, img_path, gender = wiki_data(
                name, img_dir=self.persona_dir/processed_name, fname='profile'
            )
            save(summary, dir_/'summary.txt')
            save(gender, dir_/'gender.json')

            # Otherwise it's an empty string if we fail to download an image.
            if not img_path:
                img_path = dir_/f'profile{self.backup_image.suffix}'
                shutil.copy2(self.backup_image, img_path)
            self.update_persona_dicts(processed_name)
        if return_data: return summary, img_path, gender

    def update_persona_dicts(self, processed_name, return_values=False):
        dir_ = self.persona_dir/processed_name
        summary = load(dir_/'summary.txt')
        self.name2gender[processed_name] = load(dir_/'gender.json')
        self.name2img_path[processed_name] = [p for p in dir_.iterdir()
                                              if p.stem == 'profile'][0]
        self.name2base[processed_name] = self._base_prompt.format(
            name=self.process_name(processed_name, inverse=True),
            summary=summary
        )
        if return_values:
            return Results(summary=summary,
                           img_path=self.name2img_path[processed_name],
                           gender=self.name2gender[processed_name])

    def process_name(self, name, inverse=False):
        if inverse:
            return name.replace('_', ' ').title()
        return name.lower().replace(' ', '_').replace('.', '')

    def personas(self, pretty=True, sort=True):
        names = list(self.name2base)
        if pretty: names = [self.process_name(name, True) for name in names]
        if sort: names = sorted(names)
        return names

    def kwargs(self, name='', fully_resolved=True, return_prompt=False,
               extra_kwargs=None, **kwargs):
        # Name param should be pretty version, i.e. no underscores. Only
        # needed when return_prompt is True.
        if 'prompt' in kwargs:
            raise RuntimeError(
                'Arg "prompt" should not be in query kwargs. It will be '
                'constructed within this method and passing it in will '
                'override the new version.'
            )
        kwargs = {**self._kwargs, **kwargs}
        for k, v in (extra_kwargs or {}).items():
            v_cls = type(v)
            # Make a new object instead of just using get() or setdefault
            # since the latter two methods both mutate our default kwargs.
            curr_val = v_cls(kwargs.get(k, v_cls()))
            if isinstance(v, Iterable):
                curr_val.extend(v)
            elif isinstance(v, Mapping):
                curr_val.update(v)
            else:
                raise TypeError(f'Key {k} has unrecognized type {v_cls} in '
                                '`extra_kwargs`.')
            kwargs[k] = curr_val

        if fully_resolved: kwargs = dict(bound_args(query_gpt3, [], kwargs))
            
        # Note: should this return an updated prompt? Right now it looks like 
        # it always returns the base one.
        if name and return_prompt:
            kwargs['prompt'] = self.name2base[self.process_name(name)]
        return kwargs

    def query_later(self, text):
        self.cached_query = text.strip()
        
    def query(self, text=None, debug=False, extra_kwargs=None, **kwargs):
        if not self.current_persona:
            raise RuntimeError('You must call the `start_conversation` '
                               'method before making a query.')
        
        # In the same spirit as our handling of kwargs here, passing in a text
        # arg will override a cached query if one exists.
        text = text or self.cached_query
        kwargs = self.kwargs(fully_resolved=False, return_prompt=False,
                             extra_kwargs=extra_kwargs, **kwargs)
        prompt = self.format_prompt(user_text=text)
        if debug:
            print('prompt:\n' + prompt)
            print(spacer())
            print('kwargs:\n', kwargs)
            print(spacer())
            print('fully resolved kwargs:\n',
                  dict(bound_args(query_gpt3, [], kwargs)))
            return
        
        # Update these after format_prompt() call and debug check.
        self.user_turns.append(text.strip())
        self.cached_query = ''
        
        # Log kwargs for troubleshooting purposes.
        save({'prompt': prompt, **kwargs}, self.log_path, verbose=False)
        prompt, resp = query_gpt3(prompt, **kwargs)
        self.gpt3_turns.append(resp.strip())
        # Can't just concat prompt and response anymore because prompt is not
        # the full conversation.
        return prompt, resp

    def _format_prompt(self, user_text='', do_full=False, 
                       exclude_trailing_name=False):
        if not self.current_persona:
            raise RuntimeError('No persona loaded. Have you started a '
                               'conversation?')
        if not do_full and not user_text:
            raise RuntimeError('user_text must be provided when '
                               'do_full=False.')
        
        pretty_name = self.process_name(self.current_persona, inverse=True)  
        if do_full:
            user_turns = self.user_turns
            gpt3_turns = self.gpt3_turns
        else:
            user_turns = (self.user_turns[-self.user_turn_window:]
                          + [user_text.strip()])
            gpt3_turns = self.gpt3_turns[-self.gpt3_turn_window:]
        user_turns = [f'Me: {turn}' for turn in user_turns]
        gpt3_turns = [f'{pretty_name}: {turn}' for turn in gpt3_turns]
        ordered = [user_turns, gpt3_turns]
        if len(gpt3_turns) == len(user_turns) and not do_full:
            ordered = reversed(ordered)
        interleaved = filter(None, flatten(zip_longest(*ordered)))
        # Need plus sign here: I guess implicit line continuation only works
        # for things that are explicitly strings (not output of str.join).
        prompt = f'{self.name2base[self.current_persona]}\n\n' \
                 + '\n\n'.join(interleaved)
        if exclude_trailing_name: return prompt
        return f'{prompt}\n\n{self.process_name(self.current_persona, True)}:'
    
    def format_prompt(self, user_text, exclude_trailing_name=False):
        return self._format_prompt(
            user_text, do_full=False,
            exclude_trailing_name=exclude_trailing_name
        )
    
    @property
    def full_conversation(self):
        return self._format_prompt(do_full=True, exclude_trailing_name=True)
    
    @full_conversation.setter
    def full_conversation(self):
        raise RuntimeError('full_conversation is read-only.')
        
    @full_conversation.deleter
    def full_conversation(self):
        raise RuntimeError('full_conversation is read-only.')
    
    @contextmanager
    def converse(self, name, fname='', download_if_necessary=False):
        try:
            _ = self.start_conversation(name, download_if_necessary)
            yield
        finally:
            self.end_conversation(fname=fname)

    @staticmethod
    def format_conversation(text, gpt_color='black'):
        def _format(line, color='black'):
            if not line: return line
            name, _, line = line.partition(':')
            # Bold's stop character also resets color so we need to color the
            # chunks separately.
            return colored(bold(name + ':'), color) + colored(line, color)

        if listlike(text): text = ' '.join(text)
        summary, *lines = text.splitlines()
        name = [name for name, n in
                Counter(line.split(':')[0]
                        for line in lines if ':' in line).most_common(2)
                if name != 'Me'][0]
        formatted_lines = [bold(summary)]
        prev_is_me = True
        for line in lines:
            if line.startswith(name + ':'):
                line = _format(line, gpt_color)
                prev_is_me = False
            elif line.startswith('Me: ') or prev_is_me:
                line = _format(line)
                prev_is_me = True
            formatted_lines.append(line)
        return '\n'.join(formatted_lines)

    def __contains__(self, name):
        return self.process_name(name) in self.name2base

    def __len__(self):
        return len(self.name2base)

In [483]:
conv = ProtoConversationManager(turn_window=3)

conversation: Your message must start with 'Hi {name}.'. Might want to try tweaking frequency penalty.
-------------------------------------------------------------------------------

Object loaded from data/conversation_personas/barack_obama/gender.json.
Object loaded from data/conversation_personas/joe_rogan/gender.json.
Object loaded from data/conversation_personas/kanye_west/gender.json.
Object loaded from data/conversation_personas/brandon_sanderson/gender.json.
Object loaded from data/conversation_personas/dr_seuss/gender.json.
Object loaded from data/conversation_personas/albert_einstein/gender.json.
Object loaded from data/conversation_personas/beyonce/gender.json.
Object loaded from data/conversation_personas/yann_lecun/gender.json.
Object loaded from data/conversation_personas/brendan_schaub/gender.json.


In [484]:
conv.start_conversation('Kanye West')
assert conv.user_turn_window, conv.gpt3_turn_window == (1, 1)

In [485]:
turn = 'Hi Kanye. Who is someone you\'ve met who influenced you as an artist?'
tmp = conv.format_prompt(turn)
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation with')
assert tmp.endswith('Kanye West:')

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:


In [488]:
tmp = conv.format_prompt(turn, exclude_trailing_name=True)
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation with')
assert not tmp.endswith('Kanye West:')

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?


In [489]:
print(conv.full_conversation)

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.




In [490]:
assert not conv.user_turns
assert not conv.gpt3_turns

In [491]:
assert not conv.cached_query
conv.query_later(turn)
assert conv.cached_query == turn

In [492]:
conv.query(debug=True)

prompt:
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:

-------------------------------------------------------------------------------

kwargs:
 {'engine_i': 3, 'temperature': 0.5, 'max_tokens': 250, 'frequency_penalty': 0.1, 'stop': ['Me:', 'This is a conversation with']}

-------------------------------------------------------------------------------

fully resolved kwargs:
 {'engine_i': 3, 'temperature': 0.5, 'frequency_penalty': 0.1, 'max_tokens': 250, 'logprobs': None, 'stream': Fals

In [493]:
conv.query(debug=True, engine_i=0)

prompt:
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:

-------------------------------------------------------------------------------

kwargs:
 {'engine_i': 0, 'temperature': 0.5, 'max_tokens': 250, 'frequency_penalty': 0.1, 'stop': ['Me:', 'This is a conversation with']}

-------------------------------------------------------------------------------

fully resolved kwargs:
 {'engine_i': 0, 'temperature': 0.5, 'frequency_penalty': 0.1, 'max_tokens': 250, 'logprobs': None, 'stream': Fals

In [494]:
prompt, resp = conv.query(engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people. I met a lot of people who were on my team. I met a lot of people who were on my team who were like, "Oh, you're a rapper. You're hip-hop." I met a lot of people who were like, "You're a rapper. You're R&B." I met a lot of people who were like, "You're a singer." I met a lot of people who were like, "You're a dancer." I met a lot of people who were like, "You're an artist."


In [495]:
assert len(conv.user_turns) == 1
assert len(conv.gpt3_turns) == 1

In [496]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:


In [497]:
assert not conv.cached_query
conv.query_later('What about your Dad? Was he artistic?')
assert conv.cached_query is not None

In [498]:
conv.query(engine_i=0, debug=True)

prompt:
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people. I met a lot of people who were on my team. I met a lot of people who were on my team who were like, "Oh, you're a rapper. You're hip-hop." I met a lot of people who were like, "You're a rapper. You're R&B." I met a lot of people who were like, "You're a singer." I met a lot of people who were like, "You're a dancer." I met a lot of people who were like, "You're an artist."

Me: What about your Dad? Was he arti

In [499]:
prompt, resp = conv.query(engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people. I met a lot of people who were on my team. I met a lot of people who were on my team who were like, "Oh, you're a rapper. You're hip-hop." I met a lot of people who were like, "You're a rapper. You're R&B." I met a lot of people who were like, "You're a singer." I met a lot of people who were like, "You're a dancer." I met a lot of people who were like, "You're an artist."

Me: What about your Dad? Was he artistic?

K

In [500]:
tmp = load('data/logs/conversation_query_kwargs.json')['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people. I met a lot of people who were on my team. I met a lot of people who were on my team who were like, "Oh, you're a rapper. You're hip-hop." I met a lot of people who were like, "You're a rapper. You're R&B." I met a lot of people who were like, "You're a singer." I met a lot of people who were like, "You're a dancer." I met a lot of people who were like, "You

In [501]:
prompt, resp = conv.query('So you\'ve definitely got a piece of both of them '
                          'in you.', engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people. I met a lot of people who were on my team. I met a lot of people who were on my team who were like, "Oh, you're a rapper. You're hip-hop." I met a lot of people who were like, "You're a rapper. You're R&B." I met a lot of people who were like, "You're a singer." I met a lot of people who were like, "You're a dancer." I met a lot of people who were like, "You're an artist."

Me: What about your Dad? Was he artistic?

K

In [502]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: What about your Dad? Was he artistic?

Kanye West: He was a musician. He was a singer. He was a dancer. He was an artist.

Me: So you've definitely got a piece of both of them in you.

Kanye West:


In [503]:
prompt, resp = conv.query('That\'s beautiful.', engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people. I met a lot of people who were on my team. I met a lot of people who were on my team who were like, "Oh, you're a rapper. You're hip-hop." I met a lot of people who were like, "You're a rapper. You're R&B." I met a lot of people who were like, "You're a singer." I met a lot of people who were like, "You're a dancer." I met a lot of people who were like, "You're an artist."

Me: What about your Dad? Was he artistic?

K

In [506]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')
assert len(tmp) < len(conv.full_conversation)

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: So you've definitely got a piece of both of them in you.

Kanye West: Yeah. I mean, I'm like the son of a musician and a dancer. I'm a musician and I'm a dancer.

Me: That's beautiful.

Kanye West:


## Updated convmanager

Later, I updated convmanager again to try to allow streaming mode. Need to test changes.

In [67]:
from jabberwocky.openai_utils import ConversationManager

In [68]:
conv = ConversationManager()

conversation: Your message must start with 'Hi {name}.'. Might want to try tweaking frequency penalty.
-------------------------------------------------------------------------------

Object loaded from data/conversation_personas/barack_obama/gender.json.
Object loaded from data/conversation_personas/george_lucas/gender.json.
Object loaded from data/conversation_personas/joe_rogan/gender.json.
Object loaded from data/conversation_personas/kanye_west/gender.json.
Object loaded from data/conversation_personas/brandon_sanderson/gender.json.
Object loaded from data/conversation_personas/dr_seuss/gender.json.
Object loaded from data/conversation_personas/albert_einstein/gender.json.
Object loaded from data/conversation_personas/beyonce/gender.json.
Object loaded from data/conversation_personas/yann_lecun/gender.json.
Object loaded from data/conversation_personas/sylvia_plath/gender.json.


In [69]:
conv.start_conversation('Kanye West')
assert conv.user_turn_window, conv.gpt3_turn_window == (1, 1)

In [70]:
turn = 'Hi Kanye. Who is someone you\'ve met who influenced you as an artist?'
tmp = conv.format_prompt(turn)
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation with')
assert tmp.endswith('Kanye West:')

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:


In [71]:
tmp = conv.format_prompt(turn, exclude_trailing_name=True)
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation with')
assert not tmp.endswith('Kanye West:')

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?


In [72]:
print(conv.full_conversation)

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.




In [73]:
assert not conv.user_turns
assert not conv.gpt3_turns

In [74]:
assert not conv.cached_query
conv.query_later(turn)
assert conv.cached_query == turn

In [75]:
conv.query(debug=True)

prompt:
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:

-------------------------------------------------------------------------------

kwargs:
 {'engine_i': 3, 'temperature': 0.5, 'max_tokens': 250, 'frequency_penalty': 0.1, 'stop': ['Me:', 'This is a conversation with']}

-------------------------------------------------------------------------------

fully resolved kwargs:
 {'engine_i': 3, 'temperature': 0.5, 'frequency_penalty': 0.1, 'max_tokens': 250, 'logprobs': None, 'stream': Fals

In [76]:
conv.query(debug=True, engine_i=0)

prompt:
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:

-------------------------------------------------------------------------------

kwargs:
 {'engine_i': 0, 'temperature': 0.5, 'max_tokens': 250, 'frequency_penalty': 0.1, 'stop': ['Me:', 'This is a conversation with']}

-------------------------------------------------------------------------------

fully resolved kwargs:
 {'engine_i': 0, 'temperature': 0.5, 'frequency_penalty': 0.1, 'max_tokens': 250, 'logprobs': None, 'stream': Fals

In [31]:
prompt, resp = conv.query(engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people in the music business. Some of them have been good to me, some of them have been bad, but I've met a lot of people who have been great to me. I've met a lot of people who have been great to me, and I've met a lot of people who have been bad to me.


In [35]:
assert len(conv.user_turns) == 1
assert len(conv.gpt3_turns) == 1

In [36]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:


In [37]:
assert not conv.cached_query
conv.query_later('What about your Dad? Was he artistic?')
assert conv.cached_query is not None

In [38]:
conv.query(engine_i=0, debug=True)

prompt:
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people in the music business. Some of them have been good to me, some of them have been bad, but I've met a lot of people who have been great to me. I've met a lot of people who have been great to me, and I've met a lot of people who have been bad to me.

Me: What about your Dad? Was he artistic?

Kanye West:

-------------------------------------------------------------------------------

kwargs:
 {'engine_i': 0, 'te

In [39]:
prompt, resp = conv.query(engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people in the music business. Some of them have been good to me, some of them have been bad, but I've met a lot of people who have been great to me. I've met a lot of people who have been great to me, and I've met a lot of people who have been bad to me.

Me: What about your Dad? Was he artistic?

Kanye West: My Dad was more of a businessman. He was a businessman, but he was more of an artist than a businessman. My Dad was mo

In [40]:
tmp = load('data/logs/conversation_query_kwargs.json')['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people in the music business. Some of them have been good to me, some of them have been bad, but I've met a lot of people who have been great to me. I've met a lot of people who have been great to me, and I've met a lot of people who have been bad to me.

Me: What about your Dad? Was he artistic?

Kanye West:


In [41]:
prompt, resp = conv.query('So you\'ve definitely got a piece of both of them '
                          'in you.', engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people in the music business. Some of them have been good to me, some of them have been bad, but I've met a lot of people who have been great to me. I've met a lot of people who have been great to me, and I've met a lot of people who have been bad to me.

Me: What about your Dad? Was he artistic?

Kanye West: My Dad was more of a businessman. He was a businessman, but he was more of an artist than a businessman. My Dad was mo

In [42]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: What about your Dad? Was he artistic?

Kanye West: My Dad was more of a businessman. He was a businessman, but he was more of an artist than a businessman. My Dad was more of a businessman than a musician.

Me: So you've definitely got a piece of both of them in you.

Kanye West:


In [43]:
prompt, resp = conv.query('That\'s beautiful.', engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I've met a lot of people in the music business. Some of them have been good to me, some of them have been bad, but I've met a lot of people who have been great to me. I've met a lot of people who have been great to me, and I've met a lot of people who have been bad to me.

Me: What about your Dad? Was he artistic?

Kanye West: My Dad was more of a businessman. He was a businessman, but he was more of an artist than a businessman. My Dad was mo

In [44]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')
assert len(tmp) < len(conv.full_conversation)

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: So you've definitely got a piece of both of them in you.

Kanye West: Yeah.

Me: That's beautiful.

Kanye West:


## Stream mode

In [77]:
conv = ConversationManager()

conversation: Your message must start with 'Hi {name}.'. Might want to try tweaking frequency penalty.
-------------------------------------------------------------------------------

Object loaded from data/conversation_personas/barack_obama/gender.json.
Object loaded from data/conversation_personas/george_lucas/gender.json.
Object loaded from data/conversation_personas/joe_rogan/gender.json.
Object loaded from data/conversation_personas/kanye_west/gender.json.
Object loaded from data/conversation_personas/brandon_sanderson/gender.json.
Object loaded from data/conversation_personas/dr_seuss/gender.json.
Object loaded from data/conversation_personas/albert_einstein/gender.json.
Object loaded from data/conversation_personas/beyonce/gender.json.
Object loaded from data/conversation_personas/yann_lecun/gender.json.
Object loaded from data/conversation_personas/sylvia_plath/gender.json.


In [78]:
conv.start_conversation('Kanye West')
assert conv.user_turn_window, conv.gpt3_turn_window == (1, 1)

In [79]:
turn = 'Hi Kanye. Who is someone you\'ve met who influenced you as an artist?'

In [80]:
conv.query(turn, debug=True)

prompt:
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:

-------------------------------------------------------------------------------

kwargs:
 {'engine_i': 3, 'temperature': 0.5, 'max_tokens': 250, 'frequency_penalty': 0.1, 'stop': ['Me:', 'This is a conversation with']}

-------------------------------------------------------------------------------

fully resolved kwargs:
 {'engine_i': 3, 'temperature': 0.5, 'frequency_penalty': 0.1, 'max_tokens': 250, 'logprobs': None, 'stream': Fals

In [81]:
for chunk in conv.query(turn, engine_i=0, stream=True):
    print(chunk, end='')

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:125: UserWarning: strip_output is automatically set to False when stream is True. It would be impossible to correctly reconstruct outputs otherwise.
  warnings.warn('strip_output is automatically set to False when stream '


 I was listening to him for a while, and I came out of the studio and he was talking about how it's not about the money. He was like, "It's not about the money. It's about the music." I was like, "Man, I'm a producer. It's about the music." He was like, "It's not about the money. It's about the music." So I was like, "I'm going to do that."



In [82]:
assert len(conv.user_turns) == 1
assert len(conv.gpt3_turns) == 1

In [83]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West:


In [84]:
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I was listening to him for a while, and I came out of the studio and he was talking about how it's not about the money. He was like, "It's not about the money. It's about the music." I was like, "Man, I'm a producer. It's about the music." He was like, "It's not about the money. It's about the music." So I was like, "I'm going to do that."


In [85]:
assert not conv.cached_query
conv.query_later('What about your Dad? Was he artistic?')
assert conv.cached_query is not None

In [86]:
for chunk in conv.query(engine_i=0, stream=True):
    print(chunk, end='')

 Yeah. He was a really good artist. He was a producer. He was like, "I'm going to be a producer. I'm going to be a producer." He was like, "I'm going to be a producer. I'm going to be a producer." He was like, "I'm going to be a producer."



In [88]:
conv.user_turns

["Hi Kanye. Who is someone you've met who influenced you as an artist?",
 'What about your Dad? Was he artistic?']

In [89]:
conv.gpt3_turns

['I was listening to him for a while, and I came out of the studio and he was talking about how it\'s not about the money. He was like, "It\'s not about the money. It\'s about the music." I was like, "Man, I\'m a producer. It\'s about the music." He was like, "It\'s not about the money. It\'s about the music." So I was like, "I\'m going to do that."',
 'Yeah. He was a really good artist. He was a producer. He was like, "I\'m going to be a producer. I\'m going to be a producer." He was like, "I\'m going to be a producer. I\'m going to be a producer." He was like, "I\'m going to be a producer."']

In [90]:
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I was listening to him for a while, and I came out of the studio and he was talking about how it's not about the money. He was like, "It's not about the money. It's about the music." I was like, "Man, I'm a producer. It's about the music." He was like, "It's not about the money. It's about the music." So I was like, "I'm going to do that."

Me: What about your Dad? Was he artistic?

Kanye West: Yeah. He was a really good artist. He was a produ

In [91]:
tmp = load('data/logs/conversation_query_kwargs.json')['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I was listening to him for a while, and I came out of the studio and he was talking about how it's not about the money. He was like, "It's not about the money. It's about the music." I was like, "Man, I'm a producer. It's about the music." He was like, "It's not about the money. It's about the music." So I was like, "I'm going to do that."

Me: What about your Dad? Was he artistic?



In [92]:
for chunk in conv.query('So you\'ve definitely got a piece of both of them '
                        'in you.', engine_i=0, stream=True):
    print(chunk, end='')

 Yeah.



In [93]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: What about your Dad? Was he artistic?

Kanye West: Yeah. He was a really good artist. He was a producer. He was like, "I'm going to be a producer. I'm going to be a producer." He was like, "I'm going to be a producer. I'm going to be a producer." He was like, "I'm going to be a producer."

Me: So you've definitely got a piece of both of them in you.

Kanye West:


In [94]:
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I was listening to him for a while, and I came out of the studio and he was talking about how it's not about the money. He was like, "It's not about the money. It's about the music." I was like, "Man, I'm a producer. It's about the music." He was like, "It's not about the money. It's about the music." So I was like, "I'm going to do that."

Me: What about your Dad? Was he artistic?

Kanye West: Yeah. He was a really good artist. He was a produ

In [95]:
# Check that we can use streaming mode on some queries and not on others. 
# Can't imagine a scenario where I'd want this but I'm pretty sure it works
# already anyway.
prompt, resp = conv.query('That\'s beautiful.', engine_i=0)
print(conv.format_conversation(conv.full_conversation))

This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: Hi Kanye. Who is someone you've met who influenced you as an artist?

Kanye West: I was listening to him for a while, and I came out of the studio and he was talking about how it's not about the money. He was like, "It's not about the money. It's about the music." I was like, "Man, I'm a producer. It's about the music." He was like, "It's not about the money. It's about the music." So I was like, "I'm going to do that."

Me: What about your Dad? Was he artistic?

Kanye West: Yeah. He was a really good artist. He was a produ

In [96]:
tmp = load(conv.log_path)['prompt']
print(tmp)

assert 'kanye_west' not in tmp
assert tmp.startswith('This is a conversation')
assert tmp.endswith('Kanye West:')
assert len(tmp) < len(conv.full_conversation)

Object loaded from data/logs/conversation_query_kwargs.json.
This is a conversation with Kanye West. Kanye Omari West (born June 8, 1977) is an American rapper, singer, record producer, businessman, politician, and fashion designer. He has been influential in the 21st-century development of mainstream hip hop, popular music, and popular culture in general.Born in Atlanta and raised in Chicago, West gained notoriety as a producer for Roc-A-Fella Records in the early 2000s, producing singles for several mainstream artists.

Me: So you've definitely got a piece of both of them in you.

Kanye West: Yeah.

Me: That's beautiful.

Kanye West:
